# About this program

プログラムの目的：

文学賞の中で最も有名と言える2つの賞（芥川賞・直木賞）の名前の由来である「芥川龍之介」と「直木三十五」。

二人の小説を合わせた文が作成できる最強の作家プログラム「芥川三十五」を作りたい！


プログラムの内容：
青空文庫から作家の作品を取得する

取得した小説作品のテキストデータを加工して形態素解析し、マルコフ連鎖を用いて新しい文章を生成する

# 必要ライブラリのインストール

In [1]:
!pip install beautifulsoup4
!pip install janome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 24.8 MB/s eta 0:00:00


# 青空文庫から小説文をDL＆加工するための関数定義

In [2]:
# 青空文庫からのダウンロードzip展開＆テキスト抽出

import re
import zipfile
import urllib.request
import os.path,glob
# 青空文庫のURLから小説テキストデータを得る関数
def get_flat_text_from_aozora(zip_url):
  # zipファイル名の取得
  zip_file_name = re.split(r'/', zip_url)[-1]
  print(zip_file_name)

  # 既にダウンロード済みか確認後、URLからファイルを取得
  if not os.path.exists(zip_file_name):
    print('Download URL = ',zip_url)
    data = urllib.request.urlopen(zip_url).read()
    with open(zip_file_name, mode="wb") as f:
      f.write(data)
  else:
    print('May be already exists')

  # 拡張子を除いた名前で、展開用フォルダを作成
  dir, ext = os.path.splitext(zip_file_name)
  if not os.path.exists(dir):
    os.makedirs(dir)

  # zipファイルの中身を全て、展開用フォルダに展開
  unzipped_data = zipfile.ZipFile(zip_file_name, 'r')
  unzipped_data.extractall(dir)
  unzipped_data.close()

  # zipファイルの削除
  os.remove(zip_file_name)
  # 注：展開フォルダの削除は入れていない

  # テキストファイル(.txt)の抽出
  wild_path = os.path.join(dir,'*.txt')
  # テキストファイルは原則1つ同梱。最初の1つを取得
  txt_file_path = glob.glob(wild_path)[0]

  print(txt_file_path)
  # 青空文庫はshift_jisのためデコードしてutf8にする
  binary_data = open(txt_file_path, 'rb').read()
  main_text = binary_data.decode('shift_jis')

  # 取得したutf8のテキストデータを返す
  return main_text


# 青空文庫のデータを加工して扱いやすくするコード

# 外字データ変換のための準備
# 外字変換のための対応表（jisx0213対応表）のダウンロード
# ※事前にダウンロード済みであれば飛ばしてもよい
!wget http://x0213.org/codetable/jisx0213-2004-std.txt

import re

# 外字変換のための対応表（jisx0213対応表）の読み込み
with open('jisx0213-2004-std.txt') as f:
  #ms = (re.match(r'(\d-\w{4})\s+U\+(\w{4})', l) for l in f if l[0] != '#')
  # 追加：jisx0213-2004-std.txtには5桁のUnicodeもあるため対応
  ms = (re.match(r'(\d-\w{4})\s+U\+(\w{4,5})', l) for l in f if l[0] != '#')
  gaiji_table = {m[1]: chr(int(m[2], 16)) for m in ms if m}

# 外字データの置き換えのための関数
def get_gaiji(s):
  # ※［＃「弓＋椁のつくり」、第3水準1-84-22］の形式を変換
  m = re.search(r'第(\d)水準\d-(\d{1,2})-(\d{1,2})', s)
  if m:
    key = f'{m[1]}-{int(m[2])+32:2X}{int(m[3])+32:2X}'
    return gaiji_table.get(key, s)
  # ※［＃「身＋單」、U+8EC3、56-1］の形式を変換
  m = re.search(r'U\+(\w{4})', s)
  if m:
    return chr(int(m[1], 16))
  # ※［＃二の字点、1-2-22］、［＃感嘆符二つ、1-8-75］の形式を変換
  m = re.search(r'.*?(\d)-(\d{1,2})-(\d{1,2})', s)
  if m:
    key = f'{int(m[1])+2}-{int(m[2])+32:2X}{int(m[3])+32:2X}'
    return gaiji_table.get(key, s)
  # 不明な形式の場合、元の文字列をそのまま返す
  return s

# 青空文庫の外字データ置き換え＆注釈＆ルビ除去などを行う加工関数
def flatten_aozora(text):
  # textの外字データ表記を漢字に置き換える処理
  text = re.sub(r'※［＃.+?］', lambda m: get_gaiji(m[0]), text)
  # 注釈文や、ルビなどの除去
  text = re.split(r'\-{5,}', text)[2]
  text = re.split(r'底本：', text)[0]
  text = re.sub(r'《.+?》', '', text)
  text = re.sub(r'［＃.+?］', '', text)
  text = text.strip()
  return text


# 複数ファイルのダウンロードや加工を一括実行する関数

import time
# ZIP-URLのリストから全てのデータをダウンロード＆加工する関数
def get_all_flat_text_from_zip_list(zip_list):
  all_flat_text = ""
  for zip_url in zip_list:
    # ダウンロードや解凍の失敗があり得るためTry文を使う
    # 十分なデータ量があるため数件の失敗はスキップでよい
    try:
      # 青空文庫からダウンロードする関数を実行
      aozora_dl_text = get_flat_text_from_aozora(zip_url)
      # 青空文庫のテキストを加工する関数を実行
      flat_text = flatten_aozora(aozora_dl_text)
      # 結果を追記して改行。
      all_flat_text += flat_text + ("\n")
      print(zip_url+" : 取得＆加工完了")
    except:
      # エラー時の詳細ログが出るおまじない
      import traceback
      traceback.print_exc()
      print(zip_url+" : 取得or解凍エラーのためスキップ")

    # 青空文庫サーバに負荷をかけすぎないように１秒待ってから次の小説へ
    time.sleep(1)

  # 全部がつながった大きなテキストデータを返す
  return all_flat_text



--2024-05-17 04:24:48--  http://x0213.org/codetable/jisx0213-2004-std.txt
Resolving x0213.org (x0213.org)... 59.106.19.103
Connecting to x0213.org (x0213.org)|59.106.19.103|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://x0213.org/codetable/jisx0213-2004-std.txt [following]
--2024-05-17 04:24:49--  https://x0213.org/codetable/jisx0213-2004-std.txt
Connecting to x0213.org (x0213.org)|59.106.19.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311997 (305K) [text/plain]
Saving to: ‘jisx0213-2004-std.txt’

jisx0213-2004-std.t 100%[===================>] 304.68K   212KB/s    in 1.4s    

2024-05-17 04:24:51 (212 KB/s) - ‘jisx0213-2004-std.txt’ saved [311997/311997]



# 青空文庫から小説文を取得する

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse

# <a href>タグのリンク先URL（絶対URL）を全て取得する関数
def get_a_href_list_from_url(load_url):
  html = requests.get(load_url)
  soup = BeautifulSoup(html.content, "html.parser")
  result_url_list = []
  for a_element in soup.find_all("a"):
    # 空だった場合は次のエレメントへ継続
    if a_element == None:
      continue
    # 各href属性を取得する
    link_str = a_element.get("href")
    # 空だった場合は次のエレメントへ継続
    if link_str == None:
      continue

    # 相対URLを絶対URLに加工する
    abs_url = urllib.parse.urljoin(load_url, link_str)

    # 取得した絶対URLを結果リストへ追加
    result_url_list.append(abs_url)
  return result_url_list

# 作者ページのURLを入力すると、ページを全探索してその作者の作品のzipファイルのURL一覧を返す関数
def sakusyaurl2zipurllist(sakusya_url):
  # 作者ページから出ている全てのリンク先URLを取得
  url_list_from_sakusya = get_a_href_list_from_url(sakusya_url)

  # 図書カード＝作品ごとのページ、のURLを探す
  # 末尾が、 /cards/000009/card50713.html　のような形式になっている
  tosyo_card_url_list = []
  for tosyo_card_url in url_list_from_sakusya:
    # 条件に一致する場合（cardのURLの場合）のみリストに追加
    if re.match(r'.*cards.*card.*\.html', tosyo_card_url):
      tosyo_card_url_list.append(tosyo_card_url)

  # 図書カード＝作品ごとのページ に再度スクレイピングでアクセスして、そのリンク先を全て取得
  zip_url_list = []
  for tosyo_card_url in tosyo_card_url_list:
    # 図書カードから出ている全てのリンク先URLを取得
    for zip_url in get_a_href_list_from_url(tosyo_card_url):
      # 条件に一致する場合（zipのURLの場合）のみリストに追加
      if re.match(r'.*aozora.*ruby.*\.zip', zip_url):
        zip_url_list.append( zip_url )

  # 取得したzipファイルの絶対URL一覧を返す
  return zip_url_list

# 芥川龍之介の全作品のzipファイルのURLリスト
Akutagawa_zip_list = sakusyaurl2zipurllist("https://www.aozora.gr.jp/index_pages/person879.html")
# 実際に取得できたリストを書き出す
print(Akutagawa_zip_list)
# 取得したリストの個数を書き出す
print(len(Akutagawa_zip_list))

# 直木三十五の全作品のzipファイルのURLリスト
Naoki_zip_list = sakusyaurl2zipurllist("https://www.aozora.gr.jp/index_pages/person216.html")
# 実際に取得できたリストを書き出す
print(Naoki_zip_list)
# 取得したリストの個数を書き出す
print(len(Naoki_zip_list))



['https://www.aozora.gr.jp/cards/000879/files/4872_ruby_20864.zip', 'https://www.aozora.gr.jp/cards/000879/files/16_ruby_344.zip', 'https://www.aozora.gr.jp/cards/000879/files/178_ruby_2210.zip', 'https://www.aozora.gr.jp/cards/000879/files/15_ruby_904.zip', 'https://www.aozora.gr.jp/cards/000879/files/43014_ruby_17392.zip', 'https://www.aozora.gr.jp/cards/000879/files/3804_ruby_27189.zip', 'https://www.aozora.gr.jp/cards/000879/files/21_ruby_1427.zip', 'https://www.aozora.gr.jp/cards/000879/files/43361_ruby_17690.zip', 'https://www.aozora.gr.jp/cards/000879/files/17_ruby_377.zip', 'https://www.aozora.gr.jp/cards/000879/files/14_ruby_1261.zip', 'https://www.aozora.gr.jp/cards/000879/files/1138_ruby_6111.zip', 'https://www.aozora.gr.jp/cards/000879/files/19_ruby_306.zip', 'https://www.aozora.gr.jp/cards/000879/files/73_ruby_1217.zip', 'https://www.aozora.gr.jp/cards/000879/files/20_ruby_302.zip', 'https://www.aozora.gr.jp/cards/000879/files/3827_ruby_27190.zip', 'https://www.aozora.gr.j

In [4]:
# 芥川龍之介の全データの取得＆加工
Akutagawa_all_text = get_all_flat_text_from_zip_list(Akutagawa_zip_list)
# 得た結果をファイルに書き込む
with open('Akutagawa_all_text.txt', 'w') as f:
  print(Akutagawa_all_text, file=f)
  print("★芥川ALLファイル出力完了")

# 直木三十五の全データの取得＆加工
Naoki_all_text = get_all_flat_text_from_zip_list(Naoki_zip_list)
# 得た結果をファイルに書き込む
with open('Naoki_all_text.txt', 'w') as f:
  print(Naoki_all_text, file=f)
  print("★直木ALLファイル出力完了")

# 江戸川＆コナンの両方の全テキストをつなげたファイルも作っておく
Akutagawa_Naoki_all_text = Akutagawa_all_text + Naoki_all_text
with open('Akutagawa_Naoki_all_text.txt', 'w') as f:
  print(Akutagawa_Naoki_all_text, file=f)
  print("★芥川＆直木ALLファイル出力完了")



4872_ruby_20864.zip
Download URL =  https://www.aozora.gr.jp/cards/000879/files/4872_ruby_20864.zip
4872_ruby_20864/aidokushono_insho.txt
https://www.aozora.gr.jp/cards/000879/files/4872_ruby_20864.zip : 取得＆加工完了
16_ruby_344.zip
Download URL =  https://www.aozora.gr.jp/cards/000879/files/16_ruby_344.zip
16_ruby_344/aki.txt
https://www.aozora.gr.jp/cards/000879/files/16_ruby_344.zip : 取得＆加工完了
178_ruby_2210.zip
Download URL =  https://www.aozora.gr.jp/cards/000879/files/178_ruby_2210.zip
178_ruby_2210/akutagawa_ryunosuke_kashu.txt
https://www.aozora.gr.jp/cards/000879/files/178_ruby_2210.zip : 取得＆加工完了
15_ruby_904.zip
Download URL =  https://www.aozora.gr.jp/cards/000879/files/15_ruby_904.zip
15_ruby_904/agunino_kami.txt
https://www.aozora.gr.jp/cards/000879/files/15_ruby_904.zip : 取得＆加工完了
43014_ruby_17392.zip
Download URL =  https://www.aozora.gr.jp/cards/000879/files/43014_ruby_17392.zip
43014_ruby_17392/agunino_kami.txt
https://www.aozora.gr.jp/cards/000879/files/43014_ruby_17392.zip : 

Traceback (most recent call last):
  File "<ipython-input-2-45544ad01974>", line 109, in get_all_flat_text_from_zip_list
    aozora_dl_text = get_flat_text_from_aozora(zip_url)
  File "<ipython-input-2-45544ad01974>", line 44, in get_flat_text_from_aozora
    main_text = binary_data.decode('shift_jis')
UnicodeDecodeError: 'shift_jis' codec can't decode byte 0xfa in position 520: illegal multibyte sequence


3806_ruby_27270.zip
Download URL =  https://www.aozora.gr.jp/cards/000879/files/3806_ruby_27270.zip
3806_ruby_27270/wasurerarenu_insho.txt
https://www.aozora.gr.jp/cards/000879/files/3806_ruby_27270.zip : 取得＆加工完了
★芥川ALLファイル出力完了
57315_ruby_57990.zip
Download URL =  https://www.aozora.gr.jp/cards/000216/files/57315_ruby_57990.zip
57315_ruby_57990/ooka_echizenno_dokuritsu.txt
https://www.aozora.gr.jp/cards/000216/files/57315_ruby_57990.zip : 取得＆加工完了
2518_ruby_25883.zip
Download URL =  https://www.aozora.gr.jp/cards/000216/files/2518_ruby_25883.zip
2518_ruby_25883/osakao_aruku.txt
https://www.aozora.gr.jp/cards/000216/files/2518_ruby_25883.zip : 取得＆加工完了
1069_ruby.zip
Download URL =  https://www.aozora.gr.jp/cards/000216/files/1069_ruby.zip
1069_ruby/kagiyanotsuji.txt
https://www.aozora.gr.jp/cards/000216/files/1069_ruby.zip : 取得＆加工完了
1723_ruby_24430.zip
Download URL =  https://www.aozora.gr.jp/cards/000216/files/1723_ruby_24430.zip
1723_ruby_24430/kan'ei_budokagami.txt
https://www.aozora.g

# 分かち書きを行う関数の定義

In [5]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenizerインスタンスの生成
tokenizer = Tokenizer()

# 文章を入れると、単語のリストにする関数
def make_wakati_list(input_str):
  result_list = []
  tokens = tokenizer.tokenize(input_str)
  for token in tokens:
    # 元の単語＋半角スペースを追加しているため、
    # 単語の切れ目全てに半角スペースが入る
    result_list.append(token.surface)
  return result_list


# マルコフ連鎖のデータを生成する

In [6]:
%%time

# 元ネタとなる小説のテキストデータと、
# マルコフ連鎖の単語数（チェインナンバー）を入れると、
# マルコフ連鎖用の辞書を作成する関数
def make_markov_dict(input_text_file_path, chain_number):
  # マルコフ連鎖用の辞書
  markov_dict = {}

  # readlinesで、テキストを読み込んで１行ごとにリスト化
  with open(input_text_file_path) as f:
    text_lines = f.readlines()
    # print(text_lines)

  # １行ごとに処理してマルコフ連鎖用の辞書に追記していく
  for one_line in text_lines:
    # １行ごとに、改行コードやタブなどを消す（綺麗化前処理）
    one_line = ''.join(one_line.splitlines())
    # 形態素解析して、１行を、単語リストにする
    word_list = make_wakati_list(one_line)

    # 単語リストの最初と最後に、文頭/文末を示すフラグを追加する
    word_list = ["__BOS__"] +  word_list + ["__EOS__"]

    # 最低でもchain_number+1個の単語が残っている必要があり、
    # word_listの単語数が十分な間は処理を繰り返す
    while len(word_list) > chain_number:
      # 最初の chain_number 個の単語を、辞書に登録する際のキーとする。
      key = tuple( word_list[0 : chain_number] )
      # 次に続く単語は、そのキーの次の単語
      value = word_list[chain_number]
      markov_dict

      # 初回登録の場合、そのkeyに対する空のリストを作る処理
      # 既にそのkeyに対するデータがある場合は何もしない
      markov_dict.setdefault(key, [] )

      # そのkeyに登録されているリストに、今回のvalueを追加する
      markov_dict[key].append(value)

      # リストの最初の単語を削除する
      word_list.pop(0)

  # 全ての行を処理し終わったら、完成した辞書データをリターン
  return markov_dict


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


# マルコフ連鎖で文章を生成する

In [7]:
import random

# マルコフ連鎖用の辞書と最初のキー候補リストを入れると文章を作る関数
def make_markov_sentence(markov_dict):
  # 出力用の文字列
  output_sentence = ""

  # 最大１万回ランダムに繰り返して冒頭を取得する
  key_list = list(markov_dict.keys())
  for a in range(10000):
    # 最初のキーをランダムに取得する
    key = random.choice(key_list)
    # 文頭のフラグが出るまで繰り返し
    if key[0] == "__BOS__":
      break

  # key(tuple型)を結合して文字列にして追記
  output_sentence += "".join(map(str, key))

  # 最大１万回繰り返し（通常は途中でbreakして終了）
  for a in range(10000):
    # keyに対応するvalue（次の単語候補のリスト）を取得
    value = markov_dict.get(key)
    # 例外処理：もしkeyが辞書に見つからない場合処理終了
    if value == None:
      break

    # 単語のリストから次の単語をランダムに選ぶ
    next_word = random.choice(value)
    # 文章に追記する
    output_sentence += next_word
    # 文末のフラグが出ていたら終了
    if next_word == "__EOS__":
      break

    # 既存キーの最初を除外して、next_wordをくっつけて新しいkeyに更新する
    # ※tupleの結合処理
    key = key[1:] + (next_word, )

  # 生成された文章を返す
  return output_sentence

# 「芥川＆直木」を対象にして実行

In [12]:
# 芥川＆直樹両先生の全作品からマルコフ連鎖用の辞書データを作成する
Akutagawa_Naoki_markov_dict = make_markov_dict("Akutagawa_Naoki_all_text.txt", 3)

# 何回か文章を生成してみる
for a in range(20):
  print(make_markov_sentence(Akutagawa_Naoki_markov_dict))

__BOS__もう一度新たに書き出せば、恒藤は正にその一人は僕の気の入れかたを見れば、あなたの所へ、小倉の袴に黒木綿の紋附の羽織をひっかけた、背の高い美貌の若者は、答えないで、じりじり退りながら、――その眼にも触れなかつたであらう。ヘロデの両手はいつの間にか紅毛人の作品の不道徳である。それが大名屋敷へばかり忍び込んで、盗んだ金が三百万両ずつの金がある。そのうちに突然部屋全体はもちろん、手足に水掻きのついているものはない。西洋へ来て観ると、楽なもんだ。細君は、書斎の外へ出た。人々が、十分に、延びなかった。__EOS__
__BOS__――今月も生み月になって気がついて見ると、権助が怒るのももっともです。ではさやうなら。チヤツクなどは真平御免だ。__EOS__
__BOS__「棄ておけ、棄ておけ。わしの髪はまだ白い訣ではなかつたであらう。ちらりと見た顔貌は瀬沼兵衛に紛れなかった。しかし彼等の話し声が聞えて来たように感じた。なんでもそこで呪い殺された上、僕の恋を嘲笑つてゐるかも知れない。が、後者は「何物かを以て、彼女を尋ねて来ましたが、すぐ、ぼろの出る粗悪品を輸出したりしながら、ほのかな灯でみると、何かと行く所も多いものですから、――__EOS__
__BOS__「クオラックス党を支配してゐる。が、その味から云えば、林右衛門を憎むようになったなら、やりすぎが無うて」__EOS__
__BOS__「ぶるぶるとするのう」__EOS__
__BOS__「施しじゃ、施しじゃ」__EOS__
__BOS__「彦根は譃、入れば召捕えられる所へ誰が参りましょう。そんな手軽な単純さよりも更に綿密だつたとすれば、吉川君にも、あれから、密貿易露見の暁には、見てゐれば、我々人間の国の御若君、えす・きりしとほろ上人伝」の方は私の精神をこめたとさ。__EOS__
__BOS__鬼の酋長はもう一度｜体中に漲っていたせいか、その辺に居らぬか、であった。高木は、こう云っている？」__EOS__
__BOS__「してわしはあんたはもう行ってしまわれた事はありませんか？　申してみい」__EOS__
__BOS__若い日本の旅行家は微笑したぎり、何ともそれには疲れてゐるばかりなのでございますから、考えれば考えるほど益審でたまりません。たとえばまだこう云う事を御話した女は美しい緑色の顔をまともに見つめながら、手